BM25+向量检索
知识库动态更新函数
一个reranker(使用了CrossEncoder)
增加对输入的query的处理
进一步增加易用性：1.提示词优化  2.memory

COT?

In [4]:
import jieba
import pandas as pd
import uuid
import os
import torch
import numpy as np

from rank_bm25 import BM25Okapi
from langchain_ollama import OllamaEmbeddings
from langchain_experimental.llms.ollama_functions import OllamaFunctions

from sentence_transformers import CrossEncoder


from neo4j import GraphDatabase

from dotenv import load_dotenv
load_dotenv()

llm = OllamaFunctions(model="qwen2.5", temperature=0, format="json") #可以替换成llama3

reranker = 'BAAI/bge-reranker-base'
# reranker = 'cross-encoder/ms-marco-MiniLM-L-6-v2'

# # jina-embeddings-v3
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# class HuggingfaceEmbedding:
#     def __init__(self, model):
#         self.model = SentenceTransformer(model, trust_remote_code=True)
#         self.model = self.model.to(device)
# 
#     def __call__(self, texts):
#         return self.model.encode(texts, device=device)
# 
# embeddings = HuggingfaceEmbedding("jinaai/jina-embeddings-v3")

# bge-large
class NewOllamaEmbeddings:
    def __init__(self, model):
        self.model = OllamaEmbeddings(model=model)

    def __call__(self, texts):
        return self.model.embed_query(texts)

Embedding_Model = "bge-large"
embeddings = NewOllamaEmbeddings(
            model=Embedding_Model,
        )

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 2
Python-dotenv could not parse statement starting at line 3
Python-dotenv could not parse statement starting at line 4
Python-dotenv could not parse statement starting at line 5


In [5]:
# load data
def load_excel_data(file):
    df = pd.read_excel(file)
    
    nodes = {
        "现象":df["现象"].dropna().unique(),
        "原因":df["原因"].dropna().unique(),
        "部件":df["部件"].dropna().unique(),
        "处理方法":df["处理方法"].dropna().unique(),
    }
    
    relationships = []
    for _, row in df.iterrows():
        relationships.append({
            "现象": row["现象"],
            "原因": row["原因"],
            "部件": row["部件"],
            "处理方法": row["处理方法"],
        })
        
    return nodes, relationships

file_path = "故障四元组.xlsx"
nodes, relationships = load_excel_data(file_path)

In [6]:
# 图数据库
# create graph database
class Neo4jGraphManager:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        self.embeddings = embeddings
        
    def close(self):
        self.driver.close()
        
    def clear_database(self):
        with self.driver.session() as session:
            try:
                session.run("DROP INDEX vector_index IF EXISTS")
                session.run("DROP INDEX text_index IF EXISTS")
            except Exception as e:
                print(f"warning when dropping index : {str(e)}")
            
            session.run("MATCH (n) DETACH DELETE n")
            
    def insert_node_with_embedding(self, label, properties):
        with self.driver.session() as session:
            embedding = self.embeddings(properties["text"])
            
            query = f"""
            CREATE (n:{label}{{
                name: $name,
                text: $text,
                embedding: $embedding,
                uuid: $uuid
            }})
            RETURN n.uuid AS node_id
            """
            
            result = session.run(query,{
                "name": properties["name"],
                "text": properties["text"],
                "embedding": embedding,
                "uuid": str(uuid.uuid4())
            })
            
            return result.single()["node_id"]
        
    def insert_relationship(self, start_label, start_name, end_label, end_name, relationship_type):
        with self.driver.session() as session:
            
            check_start_query = f"""
            MATCH (a:{start_label} {{name: $start_name}})
            RETURN count(a) > 0 AS node_exists
            """
            
            start_node_exists = session.run(check_start_query, {"start_name": start_name}).single()["node_exists"]
            
            if not start_node_exists:
                raise ValueError(f"start node {start_label} (name={start_name}) does not exist")
            
            check_end_query = f"""
            MATCH (a:{end_label} {{name: $end_name}})
            RETURN count(a) > 0 AS node_exists
            """
            
            end_node_exists = session.run(check_end_query, {"end_name": end_name}).single()["node_exists"]
            
            if not end_node_exists:
                raise ValueError(f"end node {end_label} (name={end_name}) does not exist")
            
            query = f"""
            MATCH (a:{start_label} {{name: $start_name}})
            MATCH (b:{end_label} {{name: $end_name}})
            MERGE (a)-[r:{relationship_type}]->(b)
            RETURN type(r) AS relationship_type
            """
            result = session.run(query, {"start_name": start_name, "end_name": end_name})
            
            return result.single()["relationship_type"]
        
    def create_indexes(self):
        # create indexes to accelerate the query for "现象"
        with self.driver.session() as session:
            try:
                session.run("""
                CREATE VECTOR INDEX vector_index IF NOT EXISTS
                FOR (n:现象)
                ON (n.embedding)
                OPTIONS {
                indexConfig: {
                    `vector.dimensions`: 1024,
                    `vector.similarity_function`: 'cosine'
                }
            }
                """)
                
                session.run("CALL db.awaitIndexes(300)")
                
            except Exception as e:
                print(f"Creating vector indexes failed : {str(e)}")
            
            try:
                session.run("""
                CREATE TEXT INDEX text_index IF NOT EXISTS
                FOR (n:现象)
                ON (n.text)
                """)
                
                session.run("CALL db.awaitIndexes(300)")
            except Exception as e:
                print(f"Creating text indexes failed : {str(e)}")
                
    def verify_indexes(self):
        with self.driver.session() as session:
            result = session.run("""
            SHOW INDEXES
            YIELD name, labelsOrTypes, properties, state
            WHERE name IN ['vector_index', 'text_index']
            RETURN name, state
            """)
            
            indexes = {record["name"]: record["state"] for record in result}
            
            missing_indexes = []
            if 'vector_index' not in indexes:
                missing_indexes.append("Vector index")
            if 'text_index' not in indexes:
                missing_indexes.append("Text index")

            if missing_indexes:
                print(f"{', '.join(missing_indexes)} missing, creating...")
                self.create_indexes()
                
            elif indexes['text_index'] != 'ONLINE':
                print(f"Vector index state: {indexes['vector_index']}, waiting...")
                session.run("CALL db.awaitIndexes(300)")


            elif indexes['vector_index'] != 'ONLINE':
                print(f"Vector index state: {indexes['vector_index']}, waiting...")
                session.run("CALL db.awaitIndexes(300)")
                
    def retrieve_with_priority(self, query:str, k=3):
        
        with self.driver.session() as session:
            print(f"[DEBUG] Query: {query}")
            embedding = self.embeddings(query)
            print(f"[DEBUG] Generated embedding for query: {embedding[:5]}... (truncated for brevity)")
            
            # contains 检索
            graph_query = """
            MATCH (n)
            WHERE n.text CONTAINS $query_text
            RETURN n.text AS text, n.name AS name, n.uuid AS uuid
            LIMIT $k
            """
            
            graph_results = session.run(graph_query, {"query_text": query, "k": k})
            graph_matches = [
                {"uuid": record["uuid"], "text": record["text"]}
                for record in graph_results
            ]
            print(f"[DEBUG] Graph database matches found: {len(graph_matches)}")
            
            # 向量检索,余弦相似度
            try:
                vector_results = session.run("""
                MATCH (n:现象)
                WHERE n.embedding IS NOT NULL
                RETURN n.text AS text, n.uuid AS uuid,
                    vector.similarity.cosine(n.embedding, $embedding) AS score
                ORDER BY score DESC
                LIMIT $k
                """,{
                    "embedding": embedding,
                    "k": k
                })
                
                vector_matches = []
                for record in vector_results:
                    vector_matches.append({
                        "uuid" : record["uuid"],
                        "text" : record["text"],
                        "score" : record["score"]
                    })
                print(f"[DEBUG] Vector search matches found: {vector_matches}")
                print(f"[DEBUG] Vector search matches found: {len(vector_matches)}")
            except Exception as e:
                print(f"[DEBUG] Vector search failed : {str(e)}")
                vector_matches = []
                
            # Remove the 'score' field from vector_matches
            for item in vector_matches:
                item.pop('score', None)
            
            # Create a set of UUIDs from graph_matches
            graph_uuids = {item['uuid'] for item in graph_matches}
            
            # Add items from graph_matches to combined_results only if their UUID is not in vector_matches
            combined_results = graph_matches + [
                item for item in vector_matches if item['uuid'] not in graph_uuids
            ]
            
            print(f"[DEBUG] Combined search results: {len(combined_results)}")
                    
            return combined_results

# construct graph database        
graph_manager = Neo4jGraphManager(uri=os.environ["NEO4J_URI"], user=os.environ["NEO4J_USERNAME"], password=os.environ["NEO4J_PASSWORD"])

graph_manager.clear_database()

for label, names in nodes.items():
    for name in names:
        graph_manager.insert_node_with_embedding(label=label, properties={"name": name, "text": name})

for rel in relationships:
    graph_manager.insert_relationship("现象", rel["现象"], "原因", rel["原因"],"原因")
    graph_manager.insert_relationship("现象", rel["现象"], "部件", rel["部件"],"涉及的部件")
    graph_manager.insert_relationship("现象", rel["现象"], "处理方法", rel["处理方法"],"处理方法")

# create index
graph_manager.create_indexes()
graph_manager.verify_indexes()

print("[DEBUG] Construction successfully finished")

graph_manager.close()
# 
# # test retriever
# graph_manager = Neo4jGraphManager(uri=os.environ["NEO4J_URI"], user=os.environ["NEO4J_USERNAME"], password=os.environ["NEO4J_PASSWORD"])
# print(graph_manager.retrieve_with_priority("天线不转动"))
# graph_manager.close()

[DEBUG] Construction successfully finished


In [7]:
class RAGPipeline:
    def __init__(self, graph_manager, llm=llm,embeddings=embeddings,reranker=reranker):
        self.graph_manager = graph_manager
        self.llm = llm
        self.embeddings = embeddings
        self.reranker = reranker
        self.history = []
    
    # 处理询问
    def refine_query(self, query):
        prompt_template = """请你从<语句>中提取<现象>，仅仅提取原话中描述故障现象的内容，不添加任何解释或改动。
            示例：
            <语句>：波导内积水如何处理
            <现象>：波导内积水
            <语句>：荧光屏不亮是怎么回事
            <现象>：荧光屏不亮
            <语句>：而且高压指示灯亮起
            <现象>：高压指示灯亮起
            
            请从语句中提取现象：
            <语句>：{query}
            <现象>：
            """
        prompt = prompt_template.format(query=query)
        try:
            # 调用LLM
            response = self.llm.invoke(prompt)
            # 假设LLM返回的JSON包含"故障现象"字段
            refined_query = response.content  # 如果提取失败，回退到原始查询
            print(f"[DEBUG] Refined Query (via LLM): {refined_query}")
        except Exception as e:
            print(f"[DEBUG] LLM refinement failed: {str(e)}, falling back to original query")
            refined_query = query  # 如果LLM调用失败，回退到原始查询
        return refined_query
            
    
    # 计算BM25分数
    def Cal_BM25(self, query, k=5):
        """
        使用 jieba 分词计算 BM25 得分
        :param query: 查询文本 (str)
        :param graph_manager: 图数据库接口
        :param k: 返回前 k 个结果
        :return: BM25 匹配结果 (list of dict)
        """
        bm25_matches = []
        with self.graph_manager.driver.session() as session:
            all_docs = session.run("MATCH (n:现象) RETURN n.text AS text")  
            # 1. 处理文档集合
            documents = [record["text"] for record in all_docs]
            # print(f"[DEBUG] Documents: {documents}")
            
            # 使用 jieba 分词处理文档
            tokenized_corpus = [list(jieba.cut(doc)) for doc in documents]  # 对每个文档分词
            # print(f"[DEBUG] Tokenized Corpus: {tokenized_corpus}")
            
            # 初始化 BM25 模型
            bm25 = BM25Okapi(tokenized_corpus)
        
        # 2. BM25 检索
        try:
            # 使用 jieba 分词处理查询
            tokenized_query = list(jieba.cut(query))
            # print(f"[DEBUG] Tokenized Query: {tokenized_query}")
            
            # 计算 BM25 得分
            bm25_scores = bm25.get_scores(tokenized_query)
            bm25_top_k = np.argsort(bm25_scores)[::-1][:k]  # 取前 k 个得分最高的文档
            
            # 存储匹配结果
            for idx in bm25_top_k:
                bm25_matches.append({
                    'text': documents[idx],
                    'bm25_score': bm25_scores[idx]
                })
                print(f"[DEBUG] BM25 search matches found: {documents[idx]} (score: {bm25_scores[idx]:.3f})")
        
        except Exception as e:
            print(f"[DEBUG] BM25 search failed: {str(e)}")
            bm25_matches = []
        
        return bm25_matches

    # 计算向量相似度
    def Cal_Similarity(self, query, k=5):
        vector_matches = []
        try:
            query_embedding = self.embeddings(query)
            print(f"[DEBUG] Query embedding: {query_embedding[:5]}...(truncated for brevity)")
            print(f"[DEBUG] Vector length: {len(query_embedding)}")
            
            with self.graph_manager.driver.session() as session:
                index_check = session.run("""
                SHOW INDEXES
                YIELD name, labelsOrTypes, properties
                WHERE name = 'vector_index'
                RETURN count(*) as count
                """)
                
                if index_check.single()['count'] == 0:
                    print(f"[DEBUG] Vector index not found, creating index...")
                    self.graph_manager.create_indexes()
                
                vector_results = session.run("""
                CALL db.index.vector.queryNodes('vector_index', $k, $embedding)
                YIELD node, score
                RETURN node.text AS text, score AS score
                ORDER BY score DESC
                """, {"k":k, "embedding":query_embedding})
                
                for record in vector_results:
                    vector_matches.append({
                        'text': record['text'],
                        'vector_score': record['score']
                    })
                    print(f"[DEBUG] Vector search matches found: {record['text']}(vector_score: {record['score']:.3f})")
                
        except Exception as e:
            print(f"[DEBUG] Vector search failed : {str(e)}") 
            vector_matches = []
        
        print(f"[DEBUG] Vector result count: {len(vector_matches)}")
        return vector_matches

    # BM25+向量混合检索
    def hybrid_retriever(self, query, bm25_weight=0.3, vector_weight=0.7, k=5):
        bm25_matches = self.Cal_BM25(query,  k=k)
        vector_matches = self.Cal_Similarity(query, k=k)
        combined_matches = {}
        for match in vector_matches:
            combined_matches[match['text']] = {'vector_score': match['vector_score'], 'bm25_score': 0.0}
        for match in bm25_matches:
            if match['text'] in combined_matches:
                combined_matches[match['text']]['bm25_score'] = match['bm25_score']
            else:
                combined_matches[match['text']] = {'vector_score': 0.0, 'bm25_score': match['bm25_score']}
    
        # 计算融合得分
        for text, scores in combined_matches.items():
            hybrid_score = bm25_weight * scores['bm25_score'] + vector_weight * scores['vector_score']
            combined_matches[text]['hybrid_score'] = hybrid_score
    
        # 转换为列表并排序
        rerank_input = [{'text': text, 'hybrid_score': scores['hybrid_score']} 
                        for text, scores in combined_matches.items()]
        reranked_matches = sorted(rerank_input, key=lambda x: x['hybrid_score'], reverse=True)
        return reranked_matches

    # 使用cross_encoder重排序
    def rerank_with_cross_encoder(self, query: str, matches: list, top_k: int = 5, alpha: int = 0.99, beta : int = 0.01) -> list:
        """
        使用中文优化的 Cross-Encoder 对混合检索结果进行重排序
        :param query: 查询文本 (中文)
        :param matches: 混合检索结果 (list of dict, 包含 'text' 和 'hybrid_score')
        :param top_k: 返回前 k 个结果
        :param alpha: cross_encoder占比
        :param beta: hybrid_score占比
        :return: 重排序后的结果
        """
        # 初始化中文 Cross-Encoder 模型
        model = CrossEncoder(self.reranker)
        
        # 准备输入：查询和候选文档对
        pairs = [[query, match['text']] for match in matches]
        # print(f"[DEBUG] Cross-Encoder input pairs: {pairs}")
        
        # 计算相关性得分
        scores = model.predict(pairs)
        # print(f"[DEBUG] Cross-Encoder scores: {scores}")
        
        # 将得分添加到 matches 中
        for match, score in zip(matches, scores):
            match['cross_encoder_score'] = float(score)  # 转换为 float 以便排序
        
        # 按 Cross-Encoder 得分排序
        reranked_matches = sorted(matches, key=lambda x: x['cross_encoder_score']*alpha + x['hybrid_score']*beta, reverse=True)
        
        print(reranked_matches[:top_k])
        # 返回前 top_k 个结果
        return reranked_matches[:top_k]
    
    # 将检索的结果处理为字符串上下文
    def format_results(self, results, K):
        response = []
        if results:
            graph_matches = [r for r in results if r['relation'] != 'vector_match']
            if graph_matches:
                response.append("检索结果:")
                for rel in graph_matches:
                    # response.append(
                    #     f"• {rel['phenomenon']}-{rel['relation']}->{rel['target']}(相关度: {rel['score']:.3f})")
                    response.append(
                        f"• {rel['phenomenon']}-{rel['relation']}->{rel['target']}")
                    
                response.append("")
            
            vector_only = [r for r in results if r['relation'] == 'vector_only']
            
            if vector_only:
                response.append("向量检索结果:")
                for rel in vector_only:
                    response.append(f"• {rel['phenomenon']}——缺少相关条目")
        
        return "\n".join(response[:min(K+1, len(response))])
    
    # 完整的检索功能
    def retriever(self, query, k=4, K=15):
        final_result = []
        
        matches = self.hybrid_retriever(query)
        reranked_matches = self.rerank_with_cross_encoder(query, matches, top_k=k)
        
        with self.graph_manager.driver.session() as session:
            for match in reranked_matches:
                text = match['text']
                score = match['cross_encoder_score']
                
                graph_results = session.run("""
                MATCH (p: 现象)-[r]->(n)
                WHERE p.text contains $text
                RETURN p.text as phenomenon, type(r) as relation, n.text as target
                """, text=text)
                
                relationships = []
                for record in graph_results:
                    relationships.append({
                        'phenomenon': record['phenomenon'],
                        'relation': record['relation'],
                        'target': record['target'],
                        'score': score
                    })
                
                if relationships:
                    final_result.extend(relationships)
                else:
                    final_result.append({
                        'phenomenon': text,
                        'relation': 'vector_match',
                        'target': 'N/A',
                        'text': text,
                        'score': score
                    })
        context = self.format_results(final_result, K)
        print("[DEGUG] context:" + context)
        return context
    
    def generate_answer(self, query, context):
        history_str = "\n".join([f"Q: {q}\nA: {a}" for q, p, a in self.history]) if self.history else "无"
        # 构造提示词
        prompt = f"""
        基于以下对话历史和检索到的相关信息回答问题：

        对话历史：
        {history_str}

        当前检索到的信息：
        {context}

        问题：
        {query}

        请提供清晰简洁的答案。注意：
        1. 如果检索到了信息，请整合信息中的原话,按照类别分条回答
        2. 如果提供的信息和对话历史不足以回答问题，请直接说明"根据现有信息无法完整回答该问题"
        3. 如果问题与对话历史相关，请结合历史信息回答

        答案：
        """
        
        # 调用 LLM
        response = llm.invoke(prompt)
        answer = response.content.strip()
        return answer
        
    def ask(self, query):
        ## 多轮问询结合
        # 拼接query
        all_query = ",".join(q for q,p,a in self.history) if self.history else ""
        all_query += query
        phenomenon = self.refine_query(query)
        phenomenons = ",".join(p for q,p,a in self.history) if self.history else "" 
        
        context = self.retriever(phenomenons + phenomenon)
        answer = self.generate_answer(all_query, context)
        
        self.history.append((query,phenomenon, answer))
        if len(self.history) > 3:  # 限制历史长度，避免上下文过长
            self.history.pop(0)
        
        return answer
    
    def check_memory(self):
        print(self.history)
    

graph_manager = Neo4jGraphManager(uri=os.environ["NEO4J_URI"], user=os.environ["NEO4J_USERNAME"],password=os.environ["NEO4J_PASSWORD"])

rag = RAGPipeline(graph_manager)
query1 = "荧光屏上没有回波是怎么回事"
answer = rag.ask(query1)
print(answer)
# query2 = "且用氛灯放在天线当中检查时氛灯辉亮"
# answer = rag.ask(query2)
# print(answer)
# query3 = "且没有混频晶体电流或者混频晶体电流很小"
# answer = rag.ask(query3)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\86159\AppData\Local\Temp\jieba.cache


[DEBUG] Refined Query (via LLM): 荧光屏上没有回波


Loading model cost 0.580 seconds.
Prefix dict has been built successfully.


[DEBUG] BM25 search matches found: 荧光屏上没有回波，且没有磁控管电流，且发射机高压指示灯辉亮 (score: 4.749)
[DEBUG] BM25 search matches found: 荧光屏上没有回波，且用氛灯放在天线当中检查时氛灯辉亮，且没有混频晶体电流或者混频晶体电流很小 (score: 3.550)
[DEBUG] BM25 search matches found: 荧光屏上没有回波，且用氛灯放在天线当中检查时氛灯辉量，且混频晶体电流正常 (score: 3.355)
[DEBUG] BM25 search matches found: 有些量程上没有扫描线 (score: 2.944)
[DEBUG] BM25 search matches found: 荧光屏上图像不规则抖动 (score: 1.453)
[DEBUG] Query embedding: [-0.028193472, -0.043359682, -0.011660766, -0.015169728, -0.0033936026]...(truncated for brevity)
[DEBUG] Vector length: 1024
[DEBUG] Vector search matches found: 荧光屏上图像不规则抖动(vector_score: 0.973)
[DEBUG] Vector search matches found: 荧光屏上出现辉亮圆(vector_score: 0.972)
[DEBUG] Vector search matches found: 荧光屏中心出现辉亮圆(vector_score: 0.943)
[DEBUG] Vector search matches found: 荧光屏上没有回波，且没有磁控管电流，且发射机高压指示灯辉亮(vector_score: 0.930)
[DEBUG] Vector search matches found: 荧光屏上没有回波，且用氛灯放在天线当中检查时氛灯辉量，且混频晶体电流正常(vector_score: 0.927)
[DEBUG] Vector result count: 5
[{'text': '荧光屏上没有回波，且没有磁控管电流，且发射机高压指示灯辉亮'

In [8]:
rag.check_memory()

[('荧光屏上没有回波是怎么回事', '荧光屏上没有回波', '荧光屏上没有回波可能由以下几个原因导致：\n1. 在个别量程，扫描末级没有电源电压。需要检查量程的扫描末级电源电压。\n2. 磁控管灯丝插头接触不良或磁控管灯丝电压数值不对。建议测量磁控管灯丝电压并插紧灯丝插座。\n3. 高压整流器没有高压输出，需检查发射机中高压整流器的输出电压，并确认高压变压器、整流硅柱和滤波电容的质量。\n4. 调制管衰老或工作状态不对。建议检查调制管各级电压特别是帘栅压，并用脉冲电压表测量预调脉冲幅度，必要时更换并重新老炼调制管。')]
